# Exchange Rates Web Scapper 💱⛏️

Zhi Chao

*Data scrapped from __https://www.x-rates.com/__*

##### Import required libraries

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import os
import sys

##### Link for HTTP GET requests

In [13]:
exc_link = "https://www.x-rates.com/historical/?from=CURR&amount=1&date=YYYYMMDD"
def generate_link( base, currency, date ):
    return base.replace("CURR",currency).replace("YYYYMMDD",date)

##### Function to scrape by day 🌞

In [14]:
def daily_exc( cur, dateobj ):
    
    # format and extract date
    date = dateobj.strftime( "%Y-%m-%d" )
    yyyy = dateobj.year
    mm = dateobj.month
    dd = dateobj.day

    # http req using link for specified currency and date
    exc_day = generate_link( exc_link, cur, date )
    reqday = requests.get( exc_day )
    soupday = BeautifulSoup( reqday.content, "html.parser" )

    # exclude top 10 currencies else they will be repeated
    all_exc = soupday.select( "td a" )[20:]
    relev_exc = { "year" : yyyy, "month" : mm, "day" : dd }

    for exc in all_exc:
        if f'from={cur}' in exc.get("href"):
            href = exc.get( "href" )
            to_cur = href.split( sep = "to=" )[1]
            relev_exc[f'{cur} to {to_cur}'] = [exc.getText()]

    return pd.DataFrame(relev_exc)

##### Function to scrape by year 📆

In [15]:
def yearly_exc( cur, year ):

    start_d = datetime.date( year, 1, 1 )
    end_d = datetime.date( year, 12, 31 )

    final = pd.DataFrame()

    while start_d <= end_d:

        exc_day = daily_exc( cur, start_d )
        final = pd.concat([final, pd.DataFrame(exc_day)])

        start_d += datetime.timedelta( days=1 )

    return final

### Testing Out! 😄😎😎

Running the code chunk below will scrape and save the data to current directory as .xlsx file

Indicate desired currency, type (daily or yearly), start/end date

**Scrapping by year might take a long time to run ❗❗❗**

##### Currency and Type

In [16]:
currency = "CNY"
type = "daily"

##### For daily

In [17]:
year = 2016
month = 2
day = 16

##### For yearly

In [18]:
start = 2022
end = 2023

##### Main function

In [19]:
if type=="yearly":
    with pd.ExcelWriter(f'./{currency} Exchange Rates {start}-{end}.xlsx', engine="openpyxl", mode="w") as writer:
        for year in range( start, end ):
            exc_year = yearly_exc( currency, year )
            exc_year.to_excel( writer, sheet_name = f'{currency} {year}', index = False )
    print("Saved")
else:
    exc_day = daily_exc(currency, datetime.date(year, month, day))
    with pd.ExcelWriter(f'./{currency} Exchange Rates {year}-{month}-{day}.xlsx', engine="openpyxl", mode="w") as writer:
        exc_day.to_excel( writer, sheet_name = f'{currency} {year}-{month}-{day}', index = False )
    print("Saved")

Saved


After saving, can visualize the data in the R Markdown file! 😄😄